# Futures trend following

In [ ]:
# Import libraries
import xarray as xr
import xarray.ufuncs as xruf
import numpy as np
import pandas as pd

import qnt.output as qnout
import qnt.ta as qnta
import qnt.data    as qndata
import qnt.stepper as qnstepper
import qnt.stats   as qnstats
import qnt.graph   as qngraph
import datetime    as dt
import plotly.graph_objs as go
import xarray.ufuncs as xruf
import time

#### Load futures data
Quantnet provides data for 39 global derivatives. 
The underlying assets are currencies, cross-rates, indices, bonds, energy and metals from the world's futures exchanges.

Suppose we want to download the data for the last 4 years. One can use the following function:

In [ ]:
fut_data = qndata.futures.load_data(tail = dt.timedelta(days = 10*365),
                        dims = ("time", "field", "asset"),
                        forward_order = True)
# The complete list
fut_data.asset

In [ ]:
# we can see historical data on a chart
trend_fig = [
    go.Scatter(
        x = fut_data.sel(asset = '6B').sel(field = 'close').to_pandas().index,
        y = fut_data.sel(asset = '6B').sel(field = 'close'),
        line = dict(width=1,color='black'))]

# draw chart
fig = go.Figure(data = trend_fig)
fig.update_yaxes(fixedrange=False) # unlock vertical scrolling
fig.show()

## Weights allocation

This function calculates positions using wma and roc as trend indicators.

In [ ]:
def calc_positions(futures, ma_periods, roc_periods, sideways_threshold):
    """ Calculates positions for given data(futures) and parameters """
    close = futures.sel(field='close')
    
    # calculate MA 
    ma = qnta.lwma(close, ma_periods)
    # calcuate ROC
    roc = qnta.roc(ma, roc_periods)

    # positive trend direction
    positive_trend = roc > sideways_threshold
    # negtive trend direction
    negative_trend = roc < -sideways_threshold 
    # sideways
    sideways_trend = abs(roc) <= sideways_threshold
    
    # We suppose that a sideways trend after a positive trend is also positive
    side_positive_trend = positive_trend.where(sideways_trend == False).ffill('time').fillna(False)
    # and a sideways trend after a negative trend is also negative
    side_negative_trend = negative_trend.where(sideways_trend == False).ffill('time').fillna(False)

    # define signals
    buy_signal = positive_trend
    buy_stop_signal = side_negative_trend

    sell_signal = negative_trend
    sell_stop_signal = side_positive_trend

    # calc positions 
    position = close.copy(True)
    position[:] = np.nan
    position = xr.where(buy_signal, 1, position)
    position = xr.where(sell_signal, -1, position)
    position = xr.where(xruf.logical_and(buy_stop_signal, position.ffill('time') > 0), 0, position)
    position = xr.where(xruf.logical_and(sell_stop_signal, position.ffill('time') < 0), 0, position)
    position = position.ffill('time').fillna(0)

    return position

This function calculate positions for multiple instruments with different parameters.

In [ ]:
def calc_output_all(data, params):
    positions = data.sel(field='close').copy(True)
    positions[:] = np.nan
    for futures_name in params.keys(): 
        p = params[futures_name]
        futures_data = data.sel(asset=futures_name).dropna('time','any')
        p = calc_positions(futures_data, p['ma_periods'], p['roc_periods'], p['sideways_threshold'])
        positions.loc[{'asset':futures_name, 'time':p.time}] = p
    
    return positions

In [ ]:
# say we select futures and their parameters for technical algorithm
params = {'F2MX': {'ma_periods': 10, 'roc_periods': 26, 'sideways_threshold': 5.0},
 'FGBX': {'ma_periods': 138, 'roc_periods': 49, 'sideways_threshold': 2.0},
 'N1U': {'ma_periods': 138, 'roc_periods': 25, 'sideways_threshold': 1.25}}

futures_list = list(params.keys())

#form the output
output = calc_output_all(fut_data.sel(asset = futures_list), params)

## Statistics

In [ ]:
def print_stat(stat):
    """Prints selected statistical key indicators:
       - the global Sharpe ratio of the strategy;
       - the global mean profit;
       - the global volatility;
       - the maximum drawdown.

       Note that Sharpe ratio, mean profit and volatility
       apply to  max simulation period, and not to the
       rolling basis of 3 years.
    """

    days = len(stat.coords["time"])
    
    returns = stat.loc[:, "relative_return"]

    equity = stat.loc[:, "equity"]
    
    sharpe_ratio = qnstats.calc_sharpe_ratio_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]

    profit = (qnstats.calc_mean_return_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    volatility = (qnstats.calc_volatility_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1])*100.0

    max_ddown = (qnstats.calc_max_drawdown(
        qnstats.calc_underwater(equity)).to_pandas().values[-1])*100.0

    print("Sharpe Ratio         : ", "{0:.3f}".format(sharpe_ratio))
    print("Mean Return [%]      : ", "{0:.3f}".format(profit))
    print("Volatility [%]       : ", "{0:.3f}".format(volatility))
    print("Maximum Drawdown [%] : ", "{0:.3f}".format(-max_ddown))

In [ ]:
stat = qnstats.calc_stat(fut_data, output, slippage_factor=0.05)
print_stat(stat)

In [ ]:
# show plot with profit and losses:
performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

# Check result

In [ ]:
qnout.check(output, fut_data)

# Write output

In [ ]:
qnout.write(output)